<a href="https://colab.research.google.com/github/HansanaNanayakkara/Deep-Learning-Mini-Project-03/blob/main/English_to_Sinhala_Translator_with_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
text_file = "/content/drive/My Drive/DL Mini project 3/English-Sinhala - Copy.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
i = 0
for line in lines:
  print(line)
  i = i + 1
  if(i==453):
    break

You|ඔබ
I|මම
Here|මෙතන
We|අප
He|ඔහු
She|ඇය
Live|සජීවි
Study|අධ්‍යයනය කරන්න
Go|යන්න
dog|බල්ලා
cat|බළලා
Ball|පන්දුව
house|නිවස
tree|ගස
run|දුවනවා
jump|පනින්න
happy|සතුටු
sad|දුක
big|ලොකු
small|කුඩා
fast|ඉක්මනින්
slow|මන්දගාමී
red|රතු
blue|නිල්
green|කොළ
yellow|කහ
eat|කනවා
drink|බොනවා
sleep|නිදාගන්නවා
wake|අවදි කරනවා
book|පොත
pen|පෑන
paper|කඩදාසි
chair|පුටුව
table|වගුව
car|මෝටර් රථය
bike|බයිසිකලය
walk|ඇවිදින්න
talk|කතා කරනවා
laugh|හිනා වෙනවා
cry|අඬනවා
smile|සිනහව
frown|රවනවා
door|දොර
window|කවුළුව
computer|පරිගණක
phone|දුරකථන
read|කියවනවා
write|ලියනවා
Sing|ගයනවා
dance|නටනවා
hear|අහන්න
see|බලන්න
touch|ස්පර්ශ කරනවා
taste|රස
smell|සුවඳ
good|යහපත
bad|නරක
right|හරි
wrong|වැරදි
up|දක්වා
down|පහළ
in|තුල
out|පිටතට
on|මත
off|අක්‍රිය
near|අසල
far|දුරින්
there|එතන
now|දැන්
then|ඉන්පසු
day|දවස
night|රෑ
sun|හිරු
moon|සඳ
star|තරුව
sky|අහස
cloud|වලාකුළු
rain|වැස්ස
snow|හිම
wind|සුළඟ
hot|උණුසුම්
cold|සීතලයි
summer|ගිම්හානය
winter|ශීත ඍතුව
spring|වසන්තය
autumn|සරත් ඍතුව
fall|වැටෙනවා
year|අවුරුදු
month|මස
w

In [4]:
text_pairs = []
for line in lines:
    if "|" in line:  # Check if the line contains the pipe character
        english, sinhala = line.split("|")
        sinhala = "[start] " + sinhala.strip() + " [end]"  # Also, strip any leading/trailing whitespace
        text_pairs.append((english.strip(), sinhala))
    else:
        print(f"Ignoring line: {line.strip()} because it does not contain a pipe character.")
        continue  # Skip to the next iteration

import random
for i in range(3):
    print(random.choice(text_pairs))



('I am happy.', '[start] මම සතුටින්. [end]')
('last', '[start] අවසන් [end]')
('contest', '[start] තරඟය [end]')


In [5]:
random.shuffle(text_pairs)

In [6]:
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]
print("Total sentences:",len(text_pairs))
print("Training set size:",len(train_pairs))
print("Validation set size:",len(val_pairs))
print("Testing set size:",len(test_pairs))

Total sentences: 427
Training set size: 299
Validation set size: 64
Testing set size: 64


In [7]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

In [8]:
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")
vocab_size = 15000
sequence_length = 20
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_sinhala_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_sinhala_texts)

In [9]:
import tensorflow as tf

batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    sin = target_vectorization(spa)
    return ({
        "english": eng,
        "sinhala": sin[:, :-1],
    }, sin[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)  # Corrected variable name
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['sinhala'].shape: {inputs['sinhala'].shape}")
    print(f"targets.shape: {targets.shape}")


inputs['english'].shape: (64, 20)
inputs['sinhala'].shape: (64, 20)
targets.shape: (64, 20)


In [10]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [11]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)



In [12]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [13]:
embed_dim = 256
dense_dim = 2048
num_heads = 8
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="sinhala")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 sinhala (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding (Posi  (None, None, 256)            3845120   ['english[0][0]']             
 tionalEmbedding)                                                                                 
                                                                                                  
 positional_embedding_1 (Po  (None, None, 256)            3845120   ['sinhala[0][0]']         

In [16]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=60, validation_data=val_ds)

Epoch 1/60
5/5 [==============================] - 8s 362ms/step - loss: 1.1037 - accuracy: 0.8855 - val_loss: 5.2030 - val_accuracy: 0.3733
Epoch 2/60
5/5 [==============================] - 0s 90ms/step - loss: 2.1071 - accuracy: 0.6951 - val_loss: 3.9075 - val_accuracy: 0.5622
Epoch 3/60
5/5 [==============================] - 0s 70ms/step - loss: 0.9126 - accuracy: 0.9396 - val_loss: 3.7883 - val_accuracy: 0.5668
Epoch 4/60
5/5 [==============================] - 0s 69ms/step - loss: 0.7015 - accuracy: 0.9725 - val_loss: 3.7705 - val_accuracy: 0.5899
Epoch 5/60
5/5 [==============================] - 0s 66ms/step - loss: 0.5976 - accuracy: 0.9762 - val_loss: 3.7755 - val_accuracy: 0.5853
Epoch 6/60
5/5 [==============================] - 0s 70ms/step - loss: 0.5255 - accuracy: 0.9789 - val_loss: 3.7736 - val_accuracy: 0.5760
Epoch 7/60
5/5 [==============================] - 0s 69ms/step - loss: 0.4711 - accuracy: 0.9826 - val_loss: 3.7820 - val_accuracy: 0.5806
Epoch 8/60
5/5 [==========

In [17]:
import numpy as np

sin_vocab = target_vectorization.get_vocabulary()
sinh_index_lookup = dict(zip(range(len(sin_vocab)), sin_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = sinh_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))


-
protection
[start] ආරක්ෂක [end]
-
question
[start] ආරක්ෂක [end]
-
enemy
[start] ආරක්ෂක [end]
-
contest
[start] ආරක්ෂක [end]
-
vice-president
[start] ආරක්ෂක [end]
-
The big tree provides shade on sunny days.
[start] බළලා නිදාගන්නවා බල්ලා එළියේ සෙල්ලම් කරනවා [end]
-
then
[start] ඉන්පසු [end]
-
hundred
[start] ආරක්ෂක [end]
-
far
[start] ආරක්ෂක [end]
-
laptop
[start] ආරක්ෂක [end]
-
Go
[start] ඔහු [end]
-
fear
[start] බිය [end]
-
enemy
[start] ආරක්ෂක [end]
-
The dog runs happily in the park.
[start] බල්ලා හයියෙන් බුරන නමුත් බළලා සන්සුන්ව සිටියි [end]
-
question
[start] ආරක්ෂක [end]
-
cashier
[start] ආරක්ෂක [end]
-
laptop
[start] ආරක්ෂක [end]
-
then
[start] ඉන්පසු [end]
-
gathering
[start] ආරක්ෂක [end]
-
eleven
[start] ආරක්ෂක [end]
